This is the base datadaset.
It includes categorical, numerical and (date, to numerical) features.
text is encoded to embeddings.
Throughout the notebook, .pt (pickles) are used, which need to be deleted in case of new data

In [0]:
dbutils.library.restartPython()

In [0]:
import os
def is_running_in_databricks() -> bool:
    return "DATABRICKS_RUNTIME_VERSION" in os.environ

In [0]:
if "DATABRICKS_RUNTIME_VERSION" in os.environ:
  #CUDA = 'cu121' 
  
  import os
  
  !pip install torch==2.1.0  torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
  import torch
  #os.environ['TORCH'] = torch.__version__
  #print(torch.__version__)
  #torch_version = '2.0.0+cu118'
  
  #!pip install pyg_lib torch_scatter torch_sparse torch_cluster -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html # torch_spline_conv
  !pip install torch_geometric
  !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html
  #!pip install torch_sparse -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  #!pip install torch_scatter -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  #!pip install pyg_lib -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  !pip install sentence-transformers
  !pip install torcheval
  !pip install matplotlib
  !pip install pandas
  !pip install tensorboard
  ROOT_FOLDER = 'dbfs:/FileStore/GraphNeuralNetworks/'
else:
  ROOT_FOLDER = ''

In [0]:
if is_running_in_databricks():
    dbutils.library.restartPython()

In [0]:
import os
def is_running_in_databricks() -> bool:
    return "DATABRICKS_RUNTIME_VERSION" in os.environ

if is_running_in_databricks():
  ROOT_FOLDER = '/dbfs/FileStore/GraphNeuralNetworks/'
else:
  ROOT_FOLDER = ''


In [0]:
import torch
print(torch.version.cuda)

In [0]:
import pandas as pd
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

In [0]:

dataset_files = [
    'final_courses_and_programs',
    'final_qualifications',
    'final_skills',

    'final_qualification_skill_edges',
    'final_course_and_program_skill_edges',
    'final_course_qualification_edges',

    'final_course_and_programs_student_edges',

    'final_people',
    'final_jobs',
    'final_organizations',
    'final_job_student_edges',
    'final_supervisor_supervisee_edges',
    'final_organization_student_edges',
    'fixed_job_job_edges',
    'fixed_job_skill_edges',
    'fixed_broader_job_job_edges',
    'final_skill_skill_edges'
    #'job_skill_edges_tfidf_100mio_50kid_800kskill'
]
df_names= []
for name in dataset_files:
    df = pd.read_csv(ROOT_FOLDER+'final_dataset_courseprograms_joined/'+name+'.csv', lineterminator='\n')
    print(name, df.shape[0])
    globals()[name.replace('final_','').replace('fixed_','')] = df
    df_names.append(name.replace('final_','').replace('fixed_',''))
    
import string
word_list = [
    "or", "up", "it", "us", "race", "location", "systems", "tools",
    "so", "addition", "id", "am", "edge"
] + list(set(list(string.ascii_lowercase))-set(['r']))
job_skill_edges = job_skill_edges.loc[~job_skill_edges['skill'].isna()]
job_skill_edges = job_skill_edges.loc[~job_skill_edges['skill'].isin(word_list)]


# filter out all skills if the whole skill is a number only
job_skill_edges = job_skill_edges.loc[~ job_skill_edges['skill'].str.isnumeric()]
print('job_skill_edges', job_skill_edges.shape[0])



course_and_program_skill_edges = course_and_program_skill_edges.loc[~course_and_program_skill_edges['Skill'].isna()]
skills = skills.loc[~skills['SKILL'].isna()]

# convert create_dte to timestamp if it is not already
x  = pd.to_datetime(courses_and_programs['CREATE_DTE'], unit='s', errors='coerce')
courses_and_programs.loc[~x.isna(), 'CREATE_DTE'] = x



courses_and_programs.loc[courses_and_programs['TITLE'].isna(), 'TITLE'] = ''
courses_and_programs.loc[courses_and_programs['DESCRIPTION'].isna(), 'DESCRIPTION'] = ''
qualifications.loc[qualifications['TITLE'].isna(), 'TITLE'] = ''
qualifications.loc[qualifications['DESCRIPTION'].isna(), 'DESCRIPTION'] = ''

In [0]:
skill_skill_edges = skill_skill_edges[['skill','related_skill','cosine_sim_score']]

In [0]:
skill_skill_edges.shape

In [0]:
# get only skill edges where skills are in the skill list
skill_skill_edges = skill_skill_edges.loc[skill_skill_edges['skill'].isin(skills['SKILL'])]
skill_skill_edges = skill_skill_edges.loc[skill_skill_edges['related_skill'].isin(skills['SKILL'])]

In [0]:
skill_skill_edges.shape

In [0]:
# add some skills which are only in the job_skill_edges but not in the skills csv
a = set(job_skill_edges['skill'].unique())
b = set(skills['SKILL'].unique())

skills_to_add = list(a-b)
skills_to_add = pd.DataFrame({'SKILL':skills_to_add})
skills = pd.concat([skills, skills_to_add], axis=0)
skills = skills.reset_index(drop=True)

In [0]:
# group by JOB_ID
job_skill_edges_grouped = job_skill_edges.groupby('JOB_ID').median()
job_skill_edges_grouped

# plot hist for n_jobdesc_used
job_skill_edges_grouped['n_jobdesc_used'].hist(bins=50, range=(0,1000))

In [0]:
# filter out jobs with less than 10 job descriptions
#job_skill_edges = job_skill_edges.loc[job_skill_edges['n_jobdesc_used']>=6]

In [0]:
# import pandas as pd
# import seaborn as sns
# import numpy as np
# import matplotlib.pyplot as plt

# # Assuming 'job_skill_edges' is your DataFrame with the column 'scaled_tfidf'

# # Calculate the threshold value (95th percentile)
# threshold = np.percentile(job_skill_edges['scaled_tfidf'], 5)

# # Replace values above the threshold with the threshold value
# #job_skill_edges.loc[job_skill_edges['scaled_tfidf'] > threshold, 'scaled_tfidf'] = threshold

# # Plot the KDE and threshold
# sns.kdeplot(data=job_skill_edges['scaled_tfidf'])
# plt.axvline(threshold, color='r', linestyle='--', label='Threshold')
# plt.legend()
# plt.xlabel('scaled_tfidf')
# plt.ylabel('Density')
# plt.title('Kernel Density Estimation')
# plt.show()

In [0]:
# for each df in df_names print the name and print the columns containing nans
for name in df_names:
    print(name)
    print(globals()[name].columns[globals()[name].isna().any()].tolist())


In [0]:
del broader_job_job_edges['Short Title']
broader_job_job_edges = broader_job_job_edges[['index_fixed','onet_index_fixed']]
broader_job_job_edges['SRC_ID'] = broader_job_job_edges['index_fixed']
broader_job_job_edges['DST_ID'] = broader_job_job_edges['onet_index_fixed']
del broader_job_job_edges['index_fixed']
del broader_job_job_edges['onet_index_fixed']

In [0]:

# for each column which contains DTE DATE or Date or date, convert the column to timestamp, for all dfs in df_names, save the highest timestamp overall to timestamp_max
cols_dte = []
timestamp_max = 0
for name in df_names:
    print(name)
    for col in globals()[name].columns:
        if 'DTE' in col or 'DATE' in col or 'Date' in col or 'date' in col:
            try:

                globals()[name][col] = (pd.to_datetime(globals()[name][col]) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
                timestamp_max = max(timestamp_max, globals()[name][col].max())
                cols_dte.append(col)
                print('>>> converted',col)
            except:
                print('could not convert', col)
                pass

# normalize all timestamps by timestamp max
for name in df_names:
    print(name)
    for col in globals()[name].columns:
        if 'DTE' in col or 'DATE' in col or 'Date' in col or 'date' in col:
            try:
                globals()[name][col] = globals()[name][col] / timestamp_max
            except:
                print('could not convert', col)
                pass



In [0]:
# get all columns not in DTE DATE or Date or date, for all df_names, if the first 20 rows of the column are not numeric, normalize the column by the max of the column
cols_numeric = []
for name in df_names:
    print(name)
    for col in globals()[name].columns:
        if col not in cols_dte and 'DTE' not in col and 'DATE' not in col and 'Date' not in col and 'date' not in col and 'ID' not in col and 'index' not in col and 'alt_title' not in col:
            if 'TOTAL' in col:
                print(col)
            try:
                if not pd.to_numeric(globals()[name][col], errors='coerce').isna().any():
                    globals()[name][col] = pd.to_numeric(globals()[name][col]) / pd.to_numeric(globals()[name][col]).max()
                    print('>>> converted', col)
                    cols_numeric.append(col)
                    
                else:
                    print('--- did not convert', col)
            except:
                print('could not convert', col)
                pass
            

In [0]:
# for all columns not in col_dte and cols_numeric, if the column only contains unique values, save it to cols_id
cols_id = []
for name in df_names:
    print(name)
    for col in globals()[name].columns:
        if col not in cols_dte and col not in cols_numeric:
            if len(globals()[name][col].unique()) == globals()[name].shape[0] or '_ID' in col or 'index_x' in col or 'index_y' in col:
                cols_id.append(col)
                print('>>> ID col', col)
            else:
                print(col)

In [0]:
# for all columns not in col_dte and cols_numeric and cols_id, if the column only contains 2 different values, save it to cols_binary and encode the column to 0 and 1
cols_binary = []
for name in df_names:
    print(name)
    for col in globals()[name].columns:
        if col not in cols_dte and col not in cols_numeric and col not in cols_id:
            if len(globals()[name][col].unique()) == 2:
                cols_binary.append(col)
                print('>>> binary col', col)
                globals()[name][col] = globals()[name][col].astype('category').cat.codes
            else:
                print(col)

In [0]:
for name in df_names:
    print(name)
    for col in globals()[name].columns:
        print('  ', col)

In [0]:
unique_domains = set()
for name in df_names:
    print(name)
    for col in globals()[name].columns:
        if 'DMN_ID' in col:
            print(col)
            unique_domains.update(globals()[name][col].unique())
            print(name, globals()[name][col].unique())
            
print('unique domain ids' , unique_domains)

# create onehot mapping for all domain ids
domain_id_to_onehot = {}
for i, domain_id in enumerate(unique_domains):
    onehot = [0] * len(unique_domains)
    onehot[i] = 1
    domain_id_to_onehot[domain_id] = onehot

# for all columns, if they are DMN_ID columns, replace the column with the onehot mapping
for name in df_names:
    print(name)
    for col in globals()[name].columns:
        if 'DMN_ID' in col:
            print(col)
            globals()[name][col] = globals()[name][col].apply(lambda x: domain_id_to_onehot[x])
            

In [0]:
# in the qualification program edges, remap the qual_id to the learning item id of the qualification
# so that we dont have to remap the skill qualification edges, which use the learning item id of the qualification

qual_to_learning_id = {row.QUAL_ID:row.LEARNING_ITEM_ID for i, row in qualifications.iterrows()}


course_qualification_edges['QUAL_LEARNING_ID'] = course_qualification_edges['QUAL_ID'].apply(lambda x: qual_to_learning_id[x])
del course_qualification_edges['QUAL_ID'] 

In [0]:

cols = {
    'courses_and_programs': {
        'categorical': [
            'CPNT_TYP_ID', 'CPNT_SRC_ID', 'LEARNING_TYPE', 'CPNT_CLASSIFICATION' #, 'DMN_ID' handle dmn id separately
        ],
        'remove':[
            'REAL_ID', 
            'THUMBNAIL_URI',
            #'TITLE_TRANSLATION_NEEDED',
            #'DESCRIPTION_TRANSLATION_NEEDED',
            #'DESC_TRANSLATION_NEEDED',
            #
        ],
        'keepadditional':[
            'TITLE_TRANSLATION_NEEDED',
            'DESCRIPTION_TRANSLATION_NEEDED',
            'DESC_TRANSLATION_NEEDED',
            'NOTACTIVE'
        ],
        'text': [
            'TITLE', 'DESCRIPTION'
        ]
    },
    'qualifications': {
        'categorical': [
            'QUAL_TYP_ID', # 'DMN_ID' handle dmn id separately
        ],
        'remove':[
            'REAL_ID', 
            #'TITLE_TRANSLATION_NEEDED',
            #'DESCRIPTION_TRANSLATION_NEEDED',
            #'DESC_TRANSLATION_NEEDED',
            #'NOTACTIVE',
            #"LEARNING_ITEM_ID"
            'QUAL_ID'
        ],
        'keepadditional':[
            'TITLE_TRANSLATION_NEEDED',
            'DESCRIPTION_TRANSLATION_NEEDED',
            'DESC_TRANSLATION_NEEDED',
            'NOTACTIVE'
        ],
        'text': [
            'TITLE', 'DESCRIPTION'
        ]
    },
    'skills':{
        'text':['SKILL']
    },
    'jobs':{
        'remove':[
            'O*NET-SOC Code', 
        ],
        'text':['TITLE']
    },
     'job_job_edges':{
        'remove':[
            'O*NET-SOC Code', 
            'Related O*NET-SOC Code',
            'Title',
            'Related Title'
            
        ],
    },
     'job_skill_edges':{
        'remove':[
            'n_jobdesc_used'
            
        ],
    },
     'qualification_skill_edges':{
        'remove':[
            'IS_APPROXIMATE'
            
        ]
        
    },
      'course_and_program_skill_edges':{
        'remove':[
            'IS_APPROXIMATE'
            
        ]
        
    },
      'people':{
          #'remove':[],
          'kepadditional':['IS_ACTIVE_INFERRED'],
          'categorical':['EMP_TYP_ID']
      }
    
}
# for all columns in cols, if the column is in the df, remove the column from the df
for name in df_names:
    print(name)
    for col in globals()[name].columns:
        if name in cols.keys() and 'remove' in cols[name].keys() and col in cols[name]['remove']:
            print('>>> remove', col)
            
            globals()[name] = globals()[name].drop(col, axis=1)

for name in df_names:
    print(name)
    print(globals()[name].columns)

In [0]:
# for all dfs for all columns, if col in cols and categorical in cols[name] and col in df, convert the column to onehot categories
onehot_and_numeric_mappings = {}
for name in df_names:
    print(name)
    onehot_and_numeric_mappings[name] = {}
    for col in globals()[name].columns:
        if name in cols.keys() and 'categorical' in cols[name].keys() and col in cols[name]['categorical']:
            print('>>> categorical', col)
            unique_categories = globals()[name][col].unique()
            to_onehot = {}
            for i, category in enumerate(unique_categories):
                onehot = [0] * len(unique_categories)
                onehot[i] = 1
                to_onehot[category] = onehot
                
            globals()[name][col] = globals()[name][col].apply(lambda x: to_onehot[x])
            print(name, col, 'categories:',str(to_onehot))
            onehot_and_numeric_mappings[name][col] = to_onehot
            
            
job_job_edges['Relatedness Tier'] = job_job_edges['Relatedness Tier'].apply(lambda x: {'Primary-Short':1,'Primary-Long':0.75, 'Supplemental':0.5}[x])
onehot_and_numeric_mappings['job_job_edges'] = {'Relatedness Tier':{'Primary-Short':1,'Primary-Long':0.75, 'Supplemental':0.5}}

In [0]:
# for all text columns apply all-mpnet-base-v2 sentence embeddings
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('all-mpnet-base-v2', device='cuda')
# for name in df_names:
#     print(name)
#     for col in globals()[name].columns:
#         if name in cols.keys() and 'text' in cols[name].keys() and col in cols[name]['text']:
#             if col =='DESCRIPTION':
#                 # skip 
#                 continue 
            
#             print('>>> text col, apply mpnet', col)
            
#             # save to pickle and if it already exists, load from pickle 
#             pickle_path = 'final_dataset_courseprograms_joined/'+name+'_'+col+'_mpnet.pkl'
#             if os.path.exists(pickle_path):
#                 if col == 'TITLE':
#                     globals()[name]['TEXT_EMBEDDING'] = pd.read_pickle(pickle_path)
#                 else:
#                     globals()[name][col] = pd.read_pickle(pickle_path)
#             else:
#                 if col == 'TITLE': # combine Title and Description
#                     globals()[name]['TEXT_EMBEDDING'] = globals()[name]['TITLE'] + '\n' + globals()[name]['DESCRIPTION']
#                     globals()[name]['TEXT_EMBEDDING'] = globals()[name]['TEXT_EMBEDDING'].apply(lambda x: model.encode(x))
#                 else:
#                     globals()[name][col] = globals()[name][col].apply(lambda x: model.encode(x))
                    
#                 globals()[name][col].to_pickle(pickle_path)
       
            

In [0]:
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import numpy as np
import os

model = SentenceTransformer('all-mpnet-base-v2', device='cuda')

batch_size = 1000  # Specify your batch size

for name in df_names:
    print(name)
    for col in globals()[name].columns:
        if name in cols.keys() and 'text' in cols[name].keys() and col in cols[name]['text']:
            if col =='DESCRIPTION':
                # skip 
                continue 
            
            print('>>> text col, apply mpnet', col)
            
            # save to pickle and if it already exists, load from pickle 
            pickle_path = ROOT_FOLDER+'final_dataset_courseprograms_joined/'+name+'_'+col+'_mpnet.pkl'
            if os.path.exists(pickle_path):
                # if col == 'TITLE':
                #     globals()[name]['TEXT_EMBEDDING'] = pd.read_pickle(pickle_path)
                # else:
                globals()[name]['TEXT_EMBEDDING'] = pd.read_pickle(pickle_path)
            else:
                num_batches = int(np.ceil(len(globals()[name][col]) / batch_size))
                embeddings = []
                
                if col == 'TITLE' and name !='jobs':
                    texts = globals()[name]['TITLE'] + '\n' + globals()[name]['DESCRIPTION']
                else:
                    texts = globals()[name][col]
                
                for i in tqdm(range(num_batches)):
                    batch_texts = texts[i * batch_size: (i + 1) * batch_size]
                    batch_embeddings = model.encode(batch_texts.tolist())
                    embeddings.extend(batch_embeddings)
                    
                # if col == 'TITLE' and name !='jobs':
                #     globals()[name]['TEXT_EMBEDDING'] = embeddings
                # else:
                globals()[name]['TEXT_EMBEDDING'] = embeddings

                pd.to_pickle(embeddings, pickle_path)
                
                
skills = skills.drop_duplicates(subset=['SKILL']) # don't want to recompute with the dropped list, so we drop afterwards

In [0]:
# job job remapping
# job skill remapping
# job broader job remapping



In [0]:
import matplotlib.pyplot as plt 
# plot job_skill_edges scaled_tfidf, histogram, from 0 to 1, 100 bins
plt.hist(job_skill_edges['scaled_tfidf'], bins=100, range=(0,0.0001))

In [0]:
# print all dfs in df_name if they have a name with edge in it
for name in df_names:
    if 'edge' in name:
        print(name)


In [0]:

# rename edges so we can do automatic mapping
# for all of the below:
# qualification_skill_edges
# course_and_program_skill_edges
# course_qualification_edges
# course_and_programs_student_edges
# job_student_edges
# supervisor_supervisee_edges
# organization_student_edges
# job_job_edges
# job_skill_edges
# broader_job_job_edges
# create a dictionary which has the column renamed for each df
rename_dict = {}
rename_dict['qualification_skill_edges'] = {'LEARNING_ITEM_ID':'qualifications_id', 'Skill':'skills_id'}
rename_dict['course_and_program_skill_edges'] = {'Skill':'skills_id','LEARNING_ITEM_ID':'courses_and_programs_id'}
rename_dict['course_qualification_edges'] = {'LEARNING_ITEM_ID':'courses_and_programs_id', 'QUAL_LEARNING_ID':'qualifications_id'}
rename_dict['course_and_programs_student_edges'] = {'LEARNING_ITEM_ID':'courses_and_programs_id', 'STUD_ID':'people_id'}
rename_dict['job_student_edges'] = {'ONET_ID':'jobs_id','STUD_ID':'people_id'}
rename_dict['supervisor_supervisee_edges'] = {'STUD_ID':'people_id1', 'SUPER_ID':'people_id2'}
rename_dict['organization_student_edges'] = {'ORG_ID':'organizations_id', 'STUD_ID':'people_id'}
rename_dict['job_job_edges'] = {'SRC_ID':'jobs_id1','DST_ID':'jobs_id2'}
rename_dict['job_skill_edges'] = {'skill':'skills_id','JOB_ID':'jobs_id'}
rename_dict['broader_job_job_edges'] = {'SRC_ID':'jobs_id1','DST_ID':'jobs_id2'}
rename_dict['skill_skill_edges'] = {'skill':'skills_id1','related_skill':'skills_id2'}

# same for the node dfs
rename_dict['courses_and_programs'] = {'LEARNING_ITEM_ID':'courses_and_programs_id'}
rename_dict['qualifications'] = {'LEARNING_ITEM_ID':'qualifications_id'}
#rename_dict['skills'] = {'SKILL':'skills_id'}
rename_dict['jobs'] = {'ID':'jobs_id'}
rename_dict['people'] = {'STUD_ID':'people_id'}
rename_dict['organizations'] = {'ORG_ID':'organizations_id'}


# rename all the columns correspondingly
for name in df_names:
    if name in rename_dict.keys():
        globals()[name] = globals()[name].rename(columns=rename_dict[name])
        


In [0]:
# print out columns for all edge types:
for name in df_names:
    if 'edge' in name:
        print(name)
        print(globals()[name].columns)

In [0]:
# in the skills df, rename the SKILL column to skill_id and create a mapping which maps the skill to the skill_id
# add the mapping to the onehot_and_numeric_mappings
# change the values in the skill_id column to the skill_id mapping
# add the original skill name as TITLE
if 'skills' in onehot_and_numeric_mappings.keys() and 'skills_id' not in onehot_and_numeric_mappings['skills'].keys() or 'skills' not in onehot_and_numeric_mappings.keys():
    skills = skills.rename(columns={'SKILL':'skills_id'})
    skills['TITLE'] = skills['skills_id']
    skill_to_skill_id = {}
    for i, skill in enumerate(skills['skills_id'].unique()):
        skill_to_skill_id[skill] = i+1
    skills['skills_id'] = skills['skills_id'].apply(lambda x: skill_to_skill_id[x])
    onehot_and_numeric_mappings['skills'] = {'skills_id':skill_to_skill_id}

# for all edge dfs in df_names, if the df has a column called skill_id, change the values in the skill_id column to the skill_id mapping
for name in df_names:
    if 'edge' in name:
        for postfix in ['', '1', '2']:
            if 'skills_id'+postfix in globals()[name].columns:
                print(name)
                globals()[name]['skills_id'+postfix] = globals()[name]['skills_id'+postfix].apply(lambda x: skill_to_skill_id[x])


In [0]:
# substract 1 from each id column
# for all df_names, if the df has a column called id, substract 1 from the column
if 'runonce1' not in globals().keys():
    runonce1 = True
    for name in df_names:
        #if 'edge' in name:
        if 'skills_id' in globals()[name].columns:
            print(name)
            globals()[name]['skills_id'] = globals()[name]['skills_id'] - 1
        if 'skills_id1' in globals()[name].columns:
            print(name)
            globals()[name]['skills_id1'] = globals()[name]['skills_id1'] - 1
        if 'skills_id2' in globals()[name].columns:
            print(name)
            globals()[name]['skills_id2'] = globals()[name]['skills_id2'] - 1
        if 'qualifications_id' in globals()[name].columns:
            print(name)
            globals()[name]['qualifications_id'] = globals()[name]['qualifications_id'] - 1
#        if name =='qualification_skill_edges':
#            globals()[name]['LEARNING_ITEM_ID'] =  globals()[name]['LEARNING_ITEM_ID'] - 1
        if 'courses_and_programs_id' in globals()[name].columns:
            print(name)
            globals()[name]['courses_and_programs_id'] = globals()[name]['courses_and_programs_id'] - 1
        if 'jobs_id' in globals()[name].columns:
            print(name)
            globals()[name]['jobs_id'] = globals()[name]['jobs_id'] - 1
        if 'jobs_id1' in globals()[name].columns:
            print(name)
            globals()[name]['jobs_id1'] = globals()[name]['jobs_id1'] - 1
        if 'jobs_id2' in globals()[name].columns:
            print(name)
            globals()[name]['jobs_id2'] = globals()[name]['jobs_id2'] - 1
        if 'people_id' in globals()[name].columns:
            print(name)
            globals()[name]['people_id'] = globals()[name]['people_id'] - 1
        if 'people_id1' in globals()[name].columns:
            print(name)
            globals()[name]['people_id1'] = globals()[name]['people_id1'] - 1
        if 'people_id2' in globals()[name].columns:
            print(name)
            globals()[name]['people_id2'] = globals()[name]['people_id2'] - 1
        if 'organizations_id' in globals()[name].columns:
            print(name)
            globals()[name]['organizations_id'] = globals()[name]['organizations_id'] - 1

    # do the same for people, skills, jobs, organizations, qualifications, courses_and_programs
    # up top

                
    # people['people_id'] = people['people_id'] - 1
    # skills['skills_id'] = skills['skills_id'] - 1
    # jobs['jobs_id1'] = jobs['jobs_id1'] - 1
    # jobs['jobs_id2'] = jobs['jobs_id2'] - 1
    # organizations['organizations_id'] = organizations['organizations_id'] - 1
    # qualifications['qualifications_id'] = qualifications['qualifications_id'] - 1
    # #qualifications['LEARNING_ITEM_ID' ] = qualifications['LEARNING_ITEM_ID'] - 1
    # courses_and_programs['courses_and_programs_id'] = courses_and_programs['courses_and_programs_id'] - 1

In [0]:
from torch_geometric.data import HeteroData
import torch
data = HeteroData()

# for all dfs which have no "edges" in the name, add the df to the data object as a node, but ignore all columns containing id or ID
for name in df_names:
    if 'edge' not in name:
        for col in globals()[name].columns:
            if col.endswith('_id'):
                # sort by the id column
                globals()[name] = globals()[name].sort_values(by=[col])



for name in df_names:
    if 'edge' not in name:
        
            
            tensor_obj = None
            print(name)
            
            
            for col in globals()[name].columns:
                
                if '_id' not in col: # lowercase only!
                    if type(globals()[name][col].values[0]) == np.ndarray or type(globals()[name][col].values[0]) == list:
                        temp_obj = torch.tensor(globals()[name][col].to_numpy().tolist())
                        print('>>> ', col, 'list')
                        print('    ', len(globals()[name][col].values[0]))
                    elif type(globals()[name][col].values[0]) == str or ('keepadditional' in cols[name] and col in cols[name]['keepadditional']):
                        data[name][col] = globals()[name][col].values
                        print('>>> ', col, 'text')
                        print('    ', type(globals()[name][col].values[0]))
                        continue # important
                        
                    else:
                        temp_obj = torch.tensor(globals()[name][col].values).reshape(-1,1)
                        print('>>> ', col, 'number')
                        print('    ', type(globals()[name][col].values[0]))
                        
                    if tensor_obj is None:
                        tensor_obj = temp_obj
                    else:
                        tensor_obj = torch.cat((tensor_obj, temp_obj), dim=1)
                    
            
            
            if tensor_obj is not None:
                print('===', tensor_obj.shape)
                data[name].x = tensor_obj
            else:
                data[name].num_nodes = globals()[name].shape[0]

# for all edge dfs, take the two columns with _id in name and add them as edge to the data object
for name in df_names:
    if 'edge' in name:
        print(name)
        print(globals()[name].columns)
        edge_cols = []
        non_id_cols = []
        for col in globals()[name].columns:
            if '_id' in col:
                edge_cols.append(col)
            else:
                non_id_cols.append(col)
                
        assert len(edge_cols) == 2, f'{name} needs 2 _id cols'
        col_name = edge_cols[0].replace('_id1','').replace('_id2','').replace('_id',''), name.replace('_edges',''), edge_cols[1].replace('_id1','').replace('_id2','').replace('_id','')
        print(name, col_name)
        data[col_name].edge_index = torch.tensor(globals()[name][edge_cols].to_numpy().T)
        if len(non_id_cols):
            data[col_name].edge_attr = torch.tensor(globals()[name][non_id_cols].to_numpy())
            
        print('>>> edge_index', edge_cols)
        print('>>> edge_attr', non_id_cols)
        
        



In [0]:
for typ in data.edge_types:
    print()
    print(typ)
    continue  
    for i in range(10000):
        
        a = data[typ].edge_index[0,i]
        b = data[typ].edge_index[1,i]
        if 'TITLE' in data[typ[0]].keys():
            print(typ[0], data[typ[0]].TITLE[a])
        if 'DESCRIPTION' in data[typ[0]].keys():
            print(typ[0], data[typ[0]].DESCRIPTION[a])
            
            
        print(typ[2])
        if 'TITLE' in data[typ[2]].keys():
            print(typ[2], data[typ[2]].TITLE[b])
        if 'DESCRIPTION' in data[typ[2]].keys():
            print(typ[2], data[typ[2]].DESCRIPTION[b])
        
        print('---')
        
        
    print('======')
        

In [0]:
data.has_self_loops(), data.has_isolated_nodes()

In [0]:
# gc collect
import gc
gc.collect()

In [0]:
import torch_geometric.transforms as T

transform = T.Compose([
       T.RemoveIsolatedNodes(),
       #T.ToUndirected(merge=False), # don't merge reversed edges into the original edge type
       T.RemoveDuplicatedEdges(),
])
data_before = data
data = transform(data)

In [0]:
for edge_type in data.edge_types:
    print(edge_type)
    print(data_before[edge_type].edge_index.shape)
    print(data[edge_type].edge_index.shape)
    print('---')

for node_type in data.node_types:
    print(node_type)
    print(data_before[node_type].num_nodes)
    print(data[node_type].num_nodes)
    print('---')
    
# ignore the ones where we have duplicate edge, idk why they are there


In [0]:

from utils.plot_degree import plot_degree
# add features:
# node degrees
from torch_geometric.utils import to_dense_adj, degree
degrees = {}
for node_type in data.node_types:
    degrees[node_type] = {}
    
for edge_type in data.edge_types:
    if edge_type[0] == edge_type[2]:
        degrees[edge_type[0]][edge_type] = degree(data[edge_type].edge_index.flatten(), num_nodes=data[edge_type[0]].num_nodes)
    else:
        degrees[edge_type[0]][edge_type] = degree(data[edge_type].edge_index[0], num_nodes=data[edge_type[0]].num_nodes)
        degrees[edge_type[2]][edge_type] = degree(data[edge_type].edge_index[1], num_nodes=data[edge_type[2]].num_nodes)

# for each node type, add the degrees of all edge types and append it as 'total_degree' to the degrees dict
for node_type in data.node_types:
    degrees[node_type]['total_degree'] = torch.zeros(data[node_type].num_nodes)
    for edge_type in degrees[node_type]:
        if edge_type != 'total_degree':
            degrees[node_type]['total_degree'] += degrees[node_type][edge_type] 

# for each node type print degree statistics for each edge type, and the total degree, max, min, mean, median. Also make a small degree plot
for node_type in data.node_types:
    print(node_type)
    # create empty torch array with length of nodes
    node_degrees = None
    
    for edge_type in degrees[node_type]:
        print(edge_type, '\nmean',degrees[node_type][edge_type].mean(), '\nstd',degrees[node_type][edge_type].std(), '\nmedian',degrees[node_type][edge_type].median(),'\nmax', degrees[node_type][edge_type].max(),'\nmin', degrees[node_type][edge_type].min())
        print('We normalize by max degree')
        #smaller figure
        x = degrees[node_type]['total_degree'].numpy()
        if node_type == edge_type[0]:
            title_ = f'{node_type}, {" ".join(edge_type[0].split("_"))}--{" ".join(edge_type[2].split("_"))}'
            save_file_ = f'{node_type}__{" ".join(edge_type[0].split("_"))}_{" ".join(edge_type[2].split("_"))}.png'
        elif edge_type == 'total_degree':
            title_ = f'{node_type}, total degree'
            save_file_ = f'{node_type}__total_degree.png'
        else:
            title_ = f'{node_type}, {" ".join(edge_type[2].split("_"))}-{" ".join(edge_type[0].split("_"))}'
            save_file_ = f'{node_type}__{" ".join(edge_type[2].split("_"))}_{" ".join(edge_type[0].split("_"))}.png'
            
        if not os.path.exists(ROOT_FOLDER+'plot_images'):
            os.mkdir(ROOT_FOLDER+'plot_images')
        plot_degree(x, title_, 'Degree', 'Count', ROOT_FOLDER+'plot_images/'+save_file_, savefig=True, xlim=[0, np.mean(x)+np.std(x)*1], color='#b8cce4')
        
        maximum = degrees[node_type][edge_type].max()
        if node_degrees is None:
            node_degrees = (degrees[node_type][edge_type]/maximum).unsqueeze(1)
        else:
            node_degrees = torch.cat((node_degrees, (degrees[node_type][edge_type]/maximum).unsqueeze(1)), dim=1)
            
        print('node degree shape',node_type, node_degrees.shape)
        
    if 'x' in data[node_type].keys():
        data[node_type].x = torch.cat((data[node_type].x, node_degrees), dim=1)
    else:
        data[node_type].x = node_degrees
        
   

In [0]:
from torch_sparse import SparseTensor
from tqdm.auto import tqdm
import torch
import gc
from torch_geometric.utils import is_undirected


''' def triangle_count(adj_matrix:SparseTensor):
    # block wise triangle count, to avoid memory issues
    diags = []
    
    row_block_size = 40
    rows = adj_matrix.size(0)
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    print('using',device)
    with torch.no_grad():
        
        adj_matrix = adj_matrix.to(device)
        counter = 0
        for block in tqdm(range(0,rows, row_block_size), desc='blockwise sparse matrix-multiplication'):
            if counter % 100 == 0:
                torch.cuda.empty_cache()
                gc.collect()
            start = block
            end = min(block+row_block_size, rows)
            adj_pow_3_block = adj_matrix[start:end].spspmm(adj_matrix).spspmm(adj_matrix)
            diag_block = get_diag(adj_pow_3_block[:,start:]).cpu()
            diags.append(diag_block)
            counter +=1

    
    return 1/2 * torch.cat(diags, dim=0) '''
def triangle_count(adj_matrix:SparseTensor):
    # adj_matmul, blockwise, so kernel does not crash
    # diag1((row1 to rowS) * full_matrix * full_matrix)
    # diag2((rowS+1 to rowR) * full_matrix * full_matrix)
    # ....
    diags = []
    
    row_block_size = 20
    rows = adj_matrix.size(0)
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    print(device)
    with torch.no_grad():
        
        adj_matrix = adj_matrix.to(device)
        count = 0
        for block in tqdm(range(0,rows, row_block_size), desc='blockwise sparse matrix-multiplication'):
            if count % 100 == 0:
                torch.cuda.empty_cache()
                gc.collect()
            start = block
            end = min(block+row_block_size, rows)
            adj_pow_2_block = adj_matrix[start:end].spspmm(adj_matrix)
            # get only the diag part now
            for i in range(0, adj_pow_2_block.size(dim=0)):
                diag_i = adj_pow_2_block[i,:].spspmm(adj_matrix[:,i]).to_dense().cpu()
                diags.append(diag_i[0,0].item())
            
            count +=1
    
    return 1/2 * torch.tensor(diags)

from torch_geometric.utils import to_undirected
from torch_sparse import SparseTensor
from torch_sparse.diag import get_diag

def undirected_triangle_counts(edge_index, max_num_nodes): 
    """Get triangles **per node**, to get count for whole graph, divide by 3"""
    
    if not is_undirected(edge_index):
        print('converting edge index to undirected')
        edge_index = to_undirected(edge_index)
    
    adj_matrix = SparseTensor(row=edge_index[0], col=edge_index[1], value=torch.ones(edge_index[1].shape[0]), sparse_sizes=(max_num_nodes, max_num_nodes))
    triangles = triangle_count(adj_matrix) 
    return triangles

In [0]:
# for edge types where both ends are the same node type, add the triangle count for the node
import torch 
if not os.path.exists(ROOT_FOLDER+'precomputed_triangles'):
    os.mkdir(ROOT_FOLDER+'precomputed_triangles')
for edge_type in data.edge_types:
    if edge_type[1] == 'broader_job_job' or edge_type[1] == 'supervisor_supervisee':
        continue # there are no triangles

    if edge_type[0] == edge_type[2]:
        print('')
        print('add triangles to', edge_type[0])
        print(edge_type)
        # if file already in pickle, load from pickle, folder is precomputed_triangles
        pickle_path = ROOT_FOLDER+'precomputed_triangles/'+edge_type[0]+'_'+('_'.join(edge_type))+'.pt'
        if os.path.exists(pickle_path):
            triangles = torch.load(pickle_path)
        else:
            triangles = undirected_triangle_counts(data[edge_type].edge_index, data[edge_type[0]].num_nodes)
            torch.save(triangles, pickle_path)
            
        print(edge_type[0], 'triangles', '\nmean', triangles.mean(),'\nstd', triangles.std(), '\nmedian',triangles.median(), '\nmax',triangles.max(), '\nmin',triangles.min())
        
        if 'x' in data[edge_type[0]].keys():
            data[edge_type[0]].x = torch.cat([data[edge_type[0]].x, triangles.unsqueeze(1)/triangles.max()], dim=1)
        else:
            data[edge_type[0]].x = triangles.unsqueeze(1)
            


print('add homogeneous triangles')
homogeneous_data = data.to_homogeneous()
if os.path.exists(ROOT_FOLDER+'precomputed_triangles/homogeneous.pt'):
    homogenous_triangles = torch.load(ROOT_FOLDER+'precomputed_triangles/homogeneous.pt')
else:
    homogenous_triangles = undirected_triangle_counts(homogeneous_data.edge_index, homogeneous_data.num_nodes)
    torch.save(homogenous_triangles, ROOT_FOLDER+'precomputed_triangles/homogeneous.pt')
    
print('homogeneous triangles', '\nmean', homogenous_triangles.mean(),'\nstd', homogenous_triangles.std(), '\nmedian',homogenous_triangles.median(), '\nmax',homogenous_triangles.max(), '\nmin',homogenous_triangles.min())

max_triangles = homogenous_triangles.max()
for i, node_type in zip(homogeneous_data.node_type.unique(), data.node_types):
    mask =  homogeneous_data.node_type == i.item()
    
    if 'x' in data[node_type].keys():
        data[node_type].x = torch.cat([data[node_type].x, homogenous_triangles[mask].unsqueeze(1)/max_triangles], dim=1)
    else:
        data[node_type].x = homogenous_triangles[mask].unsqueeze(1)

 

In [0]:
node_type

In [0]:
homogeneous_data.node_type.unique()

In [0]:
homogeneous_data

In [0]:
data.node_types

In [0]:
data['courses_and_programs'].x.shape, torch.sum(homogeneous_data.node_type==0)

In [0]:
triangles.shape

In [0]:
data = T.ToUndirected(merge=False)(data)

In [0]:
import os
import torch
from torch_geometric.data import HeteroData


filename = ROOT_FOLDER+'HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt'
if os.path.exists(filename):
    # data = HeteroData.from_dict(torch.load('./'+filename))
    raise Exception('File already exists')
else:
    torch.save(data.to_dict(), filename)

In [0]:
data

In [0]:
#data['jobs','job_job','jobs']['abc'] = 1

In [0]:
#data